First exploration of a new strategy: Inside Bar Momentum.  Simply:  if high-low of bar 2 within high-low of bar 1: signal...

In [9]:
import pandas as pd
import plotly.graph_objects as go
import utils

We know we are going to look for data on USD_JPY_H4, but we haven't yet tested collect_his_data with this pairing, so we need to run that first.  Might as well refresh while we're in the area...

In [10]:
import os


In [11]:
%run ./collect_his_data.py

400
M5 GBP_JPY
GBP_JPY M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00.000000000Z
M5 GBP_USD
GBP_USD M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00.000000000Z
M5 GBP_CAD
GBP_CAD M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00.000000000Z
M5 GBP_CHF
GBP_CHF M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00.000000000Z
M5 GBP_NZD
GBP_NZD M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00.000000000Z
M5 USD_JPY
USD_JPY M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00.000000000Z
M5 USD_CAD
USD_CAD M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00.000000000Z
M5 USD_CHF
USD_CHF M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00.000000000Z
M5 CAD_JPY
CAD_JPY M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00.000000000Z
M5 CAD_CHF
CAD_CHF M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00.000000000Z
M5 EUR_GBP
EUR_GBP M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00.000000000Z
M5 EUR_JPY
EUR_JPY M5 2020-01-01T22:00:00.000000000Z 2022-12-30T21:55:00

In [13]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_csv(utils.get_hist_data_filename(pair, granularity))
df_raw.head()

,Unnamed: 0,ticker,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c
0,0,USD_JPY,2020-01-01T22:00:00.000000000Z,920,108.601,108.733,108.586,108.667,108.651,108.761,108.607,108.673,108.701,108.808,108.616,108.679
1,1,USD_JPY,2020-01-02T02:00:00.000000000Z,579,108.664,108.751,108.652,108.736,108.671,108.758,108.658,108.742,108.678,108.764,108.664,108.747
2,2,USD_JPY,2020-01-02T06:00:00.000000000Z,1418,108.738,108.836,108.688,108.833,108.744,108.842,108.696,108.839,108.750,108.848,108.703,108.845
3,3,USD_JPY,2020-01-02T10:00:00.000000000Z,1134,108.830,108.859,108.652,108.688,108.836,108.866,108.658,108.694,108.842,108.872,108.665,108.700
4,4,USD_JPY,2020-01-02T14:00:00.000000000Z,3273,108.691,108.742,108.206,108.523,108.697,108.748,108.212,108.529,108.703,108.754,108.219,108.535
